In [1]:
import numpy as np
import pandas as pd
import json
import folium

In [2]:
bklynzips = [11212, 11213, 11216, 11233, 11238, 11209, 11214, 11228, 11204, 11218, 11219, 11230]
bklynzips_str = [str(i) for i in bklynzips]

In [73]:
#create fictitious df
temp = [np.random.randint(10,100) for i in range(12)]
df = pd.DataFrame([bklynzips_str,temp]).T

df.rename(columns={0:"zipcode",1:"temperature"},inplace=True)

In [74]:
geozips = []

with open("ny_new_york_zip_codes_geo.min.json") as jsonfile:
    data = json.load(jsonfile)
tmp = data

#remove zips we dont need
for i in range(len(tmp["features"])):
    if tmp["features"][i]["properties"]["ZCTA5CE10"] in bklynzips_str :
        geozips.append(tmp["features"][i])

#creating new JSON object (this is just to mimic the original json file heirarchy with less zipcodes)
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips

# save JSON object as updated-file
open("updated-file.json", "w").write(
    json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': '))
    )

195560

### Create HTML file map

In [53]:
def create_map(table, zips, mapped_feature):
    # reading of the updated GeoJSON file
    ny_geo = r'updated-file.json'
    # initiating a Folium map with LA's longitude and latitude
    m = folium.Map(location = [40.7128, -74.0060], zoom_start = 12)
    # creating a choropleth map
    m.choropleth(
        geo_data = ny_geo,
        fill_opacity = 0.7,
        line_opacity = 0.2,
        data = table,
        # refers to which key within the GeoJSON to map the ZIP code to
        key_on = 'feature.properties.ZCTA5CE10',
        # first element contains location information, second element contains feature of interest
        columns = [zips, mapped_feature],
        fill_color = 'Greens',
        legend_name = (' ').join(mapped_feature.split('_')).title() + ' Across NY'
    )
    folium.LayerControl().add_to(m)
    # save map with filename based on the feature of interest
    m.save(outfile = mapped_feature + '_map.html')

In [54]:
create_map(df, 'zipcode', 'temperature')

C:\Users\rayl1\AppData\Local\Continuum\anaconda3\lib\site-packages\folium\folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


### Maps in ipynb notebook

In [77]:
import geopandas as gpd

In [79]:
#create a geoDataframe
geom = gpd.read_file("updated-file.json")

#get centroid of polygon for markers
geom["centroid"] = geom.geometry.centroid

#get centroid_latlon notice the map function is in [y,x] format
geom["centroid_lonlat"] = geom["centroid"].map(lambda i: [i.y,i.x])

#create new geoDataframe with fewer columns
geodf = gpd.GeoDataFrame(df.merge(geom[["ZCTA5CE10","geometry","centroid","centroid_lonlat"]],
                          left_on="zipcode",
                          right_on="ZCTA5CE10",
                          how='inner').drop("ZCTA5CE10",axis=1))

In [94]:
row

zipcode                                                        11230
temperature                                                       88
geometry           POLYGON ((-73.972161 40.633112, -73.9719039999...
centroid                POINT (-73.96510546794168 40.62216449230305)
centroid_lonlat              [40.62216449230305, -73.96510546794168]
Name: 11, dtype: object

In [95]:
ny_geo = r'updated-file.json'

#we need the original gson file for the polygon coords
with open("ny_new_york_zip_codes_geo.min.json") as jsonfile:
    data = json.load(jsonfile)
    
#Create Map
m = folium.Map(location = [40.7128, -74.0060], zoom_start = 12)
folium.Choropleth(geo_data = ny_geo,
        fill_opacity = 0.7,
        data = df,
        # refers to which key within the GeoJSON to map the ZIP code to
        key_on = 'feature.properties.ZCTA5CE10',
        # first element contains location information, second element contains feature of interest
        columns = ["zipcode", "temperature"],
        fill_color = 'Greens',
        legend_name = (' ').join("temperature".split('_')).title() + ' Across NY').add_to(m)
for index, row in geodf.iterrows():
    #folium.Polygon(data["features"][index]["geometry"]["coordinates"], tooltip=row.zipcode).add_to(m)
    folium.Circle(location=row.centroid_lonlat,
                  radius=1,
                  tooltip=row.zipcode).add_to(m)
    
display(m)